# Multiple Nearly Optimal Solutions: Optimal Is Not Always the Only Answer

This notebook demonstrates that **optimal solutions** are not always the only answer. Multiple solutions may be nearly optimal, and sometimes nearly optimal solutions are better overall.

Understanding this helps you:
- Recognize that optimal for objectives may not be optimal overall
- Evaluate solutions beyond just objective value
- Consider implementability, robustness, and other practical factors
- Make informed choices among good solutions


## Key Concepts

**Optimal Solution**:
- The best solution according to your objectives
- Found through optimization
- May be difficult to implement or fragile

**Nearly Optimal Solutions**:
- Solutions with objective values very close to optimal (within 1-5%)
- May be easier to implement
- May be more robust to changes
- May be better overall when considering all factors

**Critical insight**: Optimal is best for your objectives, but nearly optimal may be better overall when considering implementability, robustness, and other practical factors.


## Scenario: Production Planning

You need to decide how many units of two products to produce. You want to maximize profit.

**Decision**: How many units of Product A and Product B to produce?

**Optimal solution**: Found through optimization (maximizes profit)

**Nearly optimal solutions**: Close to optimal profit but may have other advantages


## Step 1: Install Required Packages (Colab)

If you're running this notebook in Google Colab, you need to install the `pulp` package first. This cell can be skipped if running locally and the package is already installed.


In [ ]:
# Install pulp package (required for optimization)
# This is needed in Google Colab; can be skipped if already installed locally
%pip install pulp -q


## Step 2: Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value


## Step 3: Find the Optimal Solution

First, let's find the optimal solution:


In [ ]:
# Problem inputs
profit_A = 50  # Profit per unit of Product A ($)
profit_B = 40  # Profit per unit of Product B ($)

capacity = 2000  # Total production capacity (units)
labor_hours = 1500  # Available labor hours
labor_per_A = 0.5  # Labor hours per unit of A
labor_per_B = 0.8  # Labor hours per unit of B

demand_A = 1200  # Maximum demand for Product A
demand_B = 1000  # Maximum demand for Product B

# Find optimal solution
model = LpProblem("Production_Optimal", LpMaximize)

produce_A_opt = LpVariable("produce_A_opt", lowBound=0, cat='Continuous')
produce_B_opt = LpVariable("produce_B_opt", lowBound=0, cat='Continuous')

model += profit_A * produce_A_opt + profit_B * produce_B_opt, "Total_Profit"

model += produce_A_opt + produce_B_opt <= capacity, "Capacity_Limit"
model += labor_per_A * produce_A_opt + labor_per_B * produce_B_opt <= labor_hours, "Labor_Limit"
model += produce_A_opt <= demand_A, "Demand_A_Limit"
model += produce_B_opt <= demand_B, "Demand_B_Limit"

model.solve()

optimal_A = value(produce_A_opt)
optimal_B = value(produce_B_opt)
optimal_profit = value(model.objective)

print("OPTIMAL SOLUTION:")
print("=" * 60)
print(f"  Produce {optimal_A:.1f} units of Product A")
print(f"  Produce {optimal_B:.1f} units of Product B")
print(f"  Total Profit: ${optimal_profit:,.2f}")
print(f"\nThis is the OPTIMAL solution (best profit).")


## Step 4: Find Nearly Optimal Solutions

Now let's find solutions that are nearly optimal (within a few percent):


In [ ]:
# Find solutions that are within 5% of optimal
target_profit_min = optimal_profit * 0.95  # 95% of optimal (within 5%)

# Try different combinations to find nearly optimal solutions
nearly_optimal_solutions = []

# Generate candidate solutions
for a in np.arange(max(0, optimal_A - 200), min(demand_A, optimal_A + 200), 50):
    for b in np.arange(max(0, optimal_B - 200), min(demand_B, optimal_B + 200), 50):
        # Check constraints
        if (a + b <= capacity and 
            labor_per_A * a + labor_per_B * b <= labor_hours and
            a <= demand_A and b <= demand_B):
            profit = profit_A * a + profit_B * b
            if profit >= target_profit_min:
                # Calculate how close to optimal
                pct_of_optimal = (profit / optimal_profit) * 100
                nearly_optimal_solutions.append({
                    'A': a,
                    'B': b,
                    'Profit': profit,
                    '% of Optimal': pct_of_optimal,
                    'Difference from Optimal': optimal_profit - profit,
                    '% Difference': 100 - pct_of_optimal
                })

# Convert to DataFrame and sort
nearly_optimal_df = pd.DataFrame(nearly_optimal_solutions)
nearly_optimal_df = nearly_optimal_df.sort_values('Profit', ascending=False).head(10)

print("NEARLY OPTIMAL SOLUTIONS (within 5% of optimal):")
print("=" * 70)
display(nearly_optimal_df.round(2))

print(f"\nKey Insight:")
print(f"  - Multiple solutions achieve profits within 5% of optimal")
print(f"  - The differences are small (${optimal_profit - nearly_optimal_df['Profit'].min():,.2f} to ${optimal_profit - nearly_optimal_df['Profit'].max():,.2f})")
print(f"  - These solutions may have other advantages")


In [ ]:
# Evaluate top solutions on other dimensions
# For simplicity, we'll use heuristics for implementability and robustness

evaluation_results = []

# Add optimal solution
evaluation_results.append({
    'Solution': 'Optimal',
    'A': optimal_A,
    'B': optimal_B,
    'Profit': optimal_profit,
    '% of Optimal': 100.0,
    'Implementability': 'Medium',  # May require changes
    'Robustness': 'Low',  # Sensitive to changes
    'Balance': 'Low'  # Focused on one product
})

# Evaluate top 3 nearly optimal solutions
for idx, row in nearly_optimal_df.head(3).iterrows():
    a, b = row['A'], row['B']
    
    # Heuristic: Implementability (how easy to implement)
    # Solutions closer to current production are easier
    current_A, current_B = 600, 600  # Assume current production
    distance_from_current = abs(a - current_A) + abs(b - current_B)
    if distance_from_current < 100:
        implementability = 'High'
    elif distance_from_current < 300:
        implementability = 'Medium'
    else:
        implementability = 'Low'
    
    # Heuristic: Robustness (how balanced)
    # More balanced solutions are more robust
    balance_ratio = min(a, b) / max(a, b) if max(a, b) > 0 else 0
    if balance_ratio > 0.7:
        robustness = 'High'
    elif balance_ratio > 0.4:
        robustness = 'Medium'
    else:
        robustness = 'Low'
    
    evaluation_results.append({
        'Solution': f'Near-Optimal {len(evaluation_results)}',
        'A': a,
        'B': b,
        'Profit': row['Profit'],
        '% of Optimal': row['% of Optimal'],
        'Implementability': implementability,
        'Robustness': robustness,
        'Balance': 'High' if balance_ratio > 0.7 else 'Medium' if balance_ratio > 0.4 else 'Low'
    })

evaluation_df = pd.DataFrame(evaluation_results)
print("EVALUATION: Optimal vs Nearly Optimal Solutions")
print("=" * 80)
display(evaluation_df.round(2))

print("\nKey Insight:")
print("  - Optimal solution maximizes profit")
print("  - But nearly optimal solutions may be better on other dimensions")
print("  - Managerial judgment is needed to choose the best overall solution")


## Step 6: Visualize Multiple Good Solutions

Let's visualize the optimal and nearly optimal solutions:


In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# Plot optimal solution (large red star)
ax.scatter(optimal_A, optimal_B, s=500, marker='*', 
           color='red', edgecolor='darkred', linewidth=2, 
           label=f'Optimal (Profit=${optimal_profit:,.0f})', zorder=10)

# Plot nearly optimal solutions
for idx, row in nearly_optimal_df.head(5).iterrows():
    ax.scatter(row['A'], row['B'], s=200, marker='o', 
               color='green', alpha=0.6, edgecolor='darkgreen', linewidth=1.5,
               label=f"Near-Optimal ({row['% of Optimal']:.1f}%)" if idx == nearly_optimal_df.head(5).index[0] else "")

# Draw constraint lines
A_line = np.linspace(0, demand_A, 100)
B_capacity = capacity - A_line
ax.plot(A_line, B_capacity, 'r--', linewidth=1.5, alpha=0.7, label='Capacity Constraint')

B_labor = (labor_hours - labor_per_A * A_line) / labor_per_B
B_labor = np.maximum(0, B_labor)
ax.plot(A_line, B_labor, 'b--', linewidth=1.5, alpha=0.7, label='Labor Constraint')

ax.axvline(x=demand_A, color='orange', linestyle='--', linewidth=1.5, alpha=0.7)
ax.axhline(y=demand_B, color='orange', linestyle='--', linewidth=1.5, alpha=0.7)

ax.set_xlabel('Units of Product A', fontsize=12)
ax.set_ylabel('Units of Product B', fontsize=12)
ax.set_title('Multiple Good Solutions: Optimal and Nearly Optimal', 
             fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, demand_A + 100)
ax.set_ylim(0, demand_B + 100)

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("  - The red star shows the optimal solution")
print("  - The green circles show nearly optimal solutions")
print("  - Multiple good solutions exist - optimal is not the only answer")


## Summary: Multiple Nearly Optimal Solutions

**Optimal Solution**:
- Best solution according to your objectives
- Found through optimization
- May be only slightly better than nearly optimal solutions

**Nearly Optimal Solutions**:
- Solutions within 1-5% of optimal profit
- May be easier to implement
- May be more robust to changes
- May be better overall when considering all factors

**Key Insight**: 
- Optimal is best for objectives, but not always the only answer
- Multiple good solutions may exist
- Managerial judgment is needed to choose the best overall solution

**Practical Implication**:
- Don't over-rely on a single optimal solution
- Evaluate optimal and nearly optimal solutions
- Consider implementability, robustness, and other factors
- Choose the solution that is best overall, not just optimal for objectives
